In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import transformers
import torch
import os
import nltk
import re

from tqdm import tqdm
tqdm.pandas()

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
tech_df = pd.read_csv('D:/Jupyter/ah_bert/BERT_AutoencoderVA_2/Data/1113_tech_train_set_1195228.csv')
tech_df.head()

,sn,file_name,data_set,domain,subdomain,source,ko,mt,en,source_language,target_language,license,style
0,TNS20A12033,TFMACH3202001202120,기술과학,기계,의공학,한국학술정보,오븐 세척제에 포함된 피부 부식성/자극성 물질은 모두 심한 눈손상성/눈 자극성에도 ...,All skin corrosive/irritant substances contain...,All skin corrosion/irritation substances conta...,ko,en,open,문어체
1,TNS19D4694,연소전 처리를 이용한 탄소포집이 가스터빈 복합화력 플랜트의 성능에 미치는 영향,기술과학,전기,에너지,한국학술정보,국내에서는 2030년 BAU 대비 37%의 온실가스 감축을 의무화함으로써 이에 따른...,"In Korea, it is urgent to come up with a solut...","In Korea, it really is urgent to produce an an...",ko,en,open,문어체
2,MDS29B1055,Original Article Serum Anti-Mullerian Hormone ...,기술과학,의학,가정의학,한국학술정보,수술 전 정맥혈을 채취하고 월경주기의 난포기 동안 8시간 이상 금식한 후 정맥혈액을...,"Venous blood was collected before surgery, and...","Before the operation, blood samples were obtai...",en,ko,open,문어체
3,MDS21A6488,장기요양 인정자의 최초 재가급여 선택과 유지 및 이탈에 대한 영향요인 Aging i...,기술과학,의학,가정의학,한국학술정보,3등급 이하 인정자는 8.2%만이 재가급여에서 시설급여로 변경하였다.,Only 8.2% of those accredited for grade 3 or l...,Only 8.2% of third graders or lower changed fr...,ko,en,open,문어체
4,TNS01V13152,특허_16주_1020170079798,기술과학,기계,열처리,특허정보원,센싱에 의해 공기 중 오일입자가 기준치이하인지 여부를 판단할 수 있다.,"In addition, it is possible to determine wheth...",It is possible to determine whether or not the...,ko,en,open,문어체


In [4]:
len(tech_df)

1195228

In [9]:
df = tech_df[['domain','mt','ko']]
df.head()

,domain,mt,ko
0,기계,All skin corrosive/irritant substances contain...,오븐 세척제에 포함된 피부 부식성/자극성 물질은 모두 심한 눈손상성/눈 자극성에도 ...
1,전기,"In Korea, it is urgent to come up with a solut...",국내에서는 2030년 BAU 대비 37%의 온실가스 감축을 의무화함으로써 이에 따른...
2,의학,"Venous blood was collected before surgery, and...",수술 전 정맥혈을 채취하고 월경주기의 난포기 동안 8시간 이상 금식한 후 정맥혈액을...
3,의학,Only 8.2% of those accredited for grade 3 or l...,3등급 이하 인정자는 8.2%만이 재가급여에서 시설급여로 변경하였다.
4,기계,"In addition, it is possible to determine wheth...",센싱에 의해 공기 중 오일입자가 기준치이하인지 여부를 판단할 수 있다.


In [10]:
domain = df.iloc[:, 0:1]
domain

,domain
0,기계
1,전기
2,의학
3,의학
4,기계
...,...
1195223,의학
1195224,기계
1195225,전기
1195226,ICT


In [7]:
# final_tech_data = tech_df_enko.iloc[:100000, :]
# final_tech_data

In [8]:
# df = pd.concat([social_df_enko, tech_df_enko])

# New

tech_df_enko

In [19]:
ict_df = df['domain'] == 'ICT'

ict_df = df[ict_df]

ict_df

,domain,mt,ko
6,ICT,The predetermined input may be an input that f...,미리 정해진 입력은 필기 입력의 일부를 포함하는 폐곡선을 형성하는 입력일 수 있다.
8,ICT,According to another aspect of the present inv...,"본 발명의 또 다른 양상에 따르면, 휴대 장치를 제어하는 장치가 제공된다."
11,ICT,The storage device 100 may communicate with th...,스토리지 장치(100)는 다양한 인터페이스를 통하여 호스트(200)와 통신할 수 있다.
16,ICT,Self-employment can be seen as a very importan...,"고용인 4명 중 1명꼴로 자영업에 의존하고 있는 만큼, 자영업은 우리에게 매우 중요..."
17,ICT,"Although not shown in the drawing, a discharge...","도면에는 도시되지 않았으나, 제1 및 제2 노드들(N1 및 N2)에는 디스차지 회로..."
...,...,...,...
1195202,ICT,The r potential codewords have the highest rel...,r 개의 포텐셜 코드워드는 후보 경로를 생성하기 위해 가장 높은 신뢰성을 가진다.
1195205,ICT,The secure subsystem is sometimes referred to ...,"보안 서브시스템은 때때로 그린으로 지칭되는 반면에, 비보안 서브시스템은 레드로 지칭된다."
1195212,ICT,"For example, a plan view of conductive layers ...","예시적으로, 메모리 블록(BLKb)의 도전층들의 평면도가 도 32에 도시되어 있다."
1195220,ICT,"That is, 405 molecular structures of all possi...","즉, 14개의 2차원 그래프로부터 변환된 가능한 모든 조합의 분자구조는 405개이다."


In [20]:
ict_df_1= ict_df.iloc[:20000, :]
ict_df_1

,domain,mt,ko
6,ICT,The predetermined input may be an input that f...,미리 정해진 입력은 필기 입력의 일부를 포함하는 폐곡선을 형성하는 입력일 수 있다.
8,ICT,According to another aspect of the present inv...,"본 발명의 또 다른 양상에 따르면, 휴대 장치를 제어하는 장치가 제공된다."
11,ICT,The storage device 100 may communicate with th...,스토리지 장치(100)는 다양한 인터페이스를 통하여 호스트(200)와 통신할 수 있다.
16,ICT,Self-employment can be seen as a very importan...,"고용인 4명 중 1명꼴로 자영업에 의존하고 있는 만큼, 자영업은 우리에게 매우 중요..."
17,ICT,"Although not shown in the drawing, a discharge...","도면에는 도시되지 않았으나, 제1 및 제2 노드들(N1 및 N2)에는 디스차지 회로..."
...,...,...,...
86081,ICT,Transmitter 3300 can transmit and receive data...,통신 인터페이스(3320)를 통해 송신기(3300)는 데이터를 송신하고 그리고 수신...
86082,ICT,Since this study investigates the effect of th...,본 연구는 스마트 밴드의 디자인 특성이 행동 변화에 미치는 영향을 조사하고 있기 때...
86083,ICT,"Alternatively, the additional member ADM may b...","또는, 추가 부재(ADM)는 제1 회로 기판(MCB)에 연결되는 전원 공급 부재일 ..."
86088,ICT,"In this case, the control circuit 40 may repea...","이때, 제어 회로(40)는 소정의 주기마다 반복적으로 센싱 데이터 분포를 생성할 수..."


In [21]:
ele_df = df['domain'] == '전기'

ele_df = df[ele_df]

ele_df

,domain,mt,ko
1,전기,"In Korea, it is urgent to come up with a solut...",국내에서는 2030년 BAU 대비 37%의 온실가스 감축을 의무화함으로써 이에 따른...
55,전기,The configurations of the electrodes prepared ...,"상기 실시예 1, 2 와 비교예 1, 2에서 제조된 전극의 구성은 하기와 같다."
59,전기,In order to give the windshield a heating func...,"자동차 앞유리에 발열 기능을 부여하기 위해서는 광투과율이 높아야 하며, 빛의 간섭이..."
66,전기,The lower gate contact plug 252 may include at...,"상기 하부 게이트 콘택 플러그(252)는 금속, 금속 합금, 및 실리사이드 중에서 ..."
74,전기,"When the battery is charged, the voltage incre...",전지는 충전하면 잔존용량 증가와 함께 전압이 상승하고 방전 시에는 반대로 전압이 감...
...,...,...,...
1195187,전기,The solid ion conductor may additionally inclu...,상기 고체이온전도체는 원자량 30 이상의 1족 원소를 추가적으로 포함할 수 있다.
1195206,전기,Plasma initial ignition may be performed using...,연결몸체(411c)에 권선된 유도 코일(442)을 이용하여 플라즈마 초기 점화를 수...
1195207,전기,"In other words, the pitch between adjacent mea...","환언하면, 인접하는 측정용 화소(28) 간의 피치가 멀티 빔의 각 빔 간의 피치가 된다."
1195221,전기,The reflective layer may be made of a metal ma...,"반사층은 구리(Cu), 은(Ag), 팔라듐(Pd) 등과 같은 금속 물질로 이루어질 ..."


In [22]:
ele_df_1= ele_df.iloc[:20000, :]
ele_df_1

,domain,mt,ko
1,전기,"In Korea, it is urgent to come up with a solut...",국내에서는 2030년 BAU 대비 37%의 온실가스 감축을 의무화함으로써 이에 따른...
55,전기,The configurations of the electrodes prepared ...,"상기 실시예 1, 2 와 비교예 1, 2에서 제조된 전극의 구성은 하기와 같다."
59,전기,In order to give the windshield a heating func...,"자동차 앞유리에 발열 기능을 부여하기 위해서는 광투과율이 높아야 하며, 빛의 간섭이..."
66,전기,The lower gate contact plug 252 may include at...,"상기 하부 게이트 콘택 플러그(252)는 금속, 금속 합금, 및 실리사이드 중에서 ..."
74,전기,"When the battery is charged, the voltage incre...",전지는 충전하면 잔존용량 증가와 함께 전압이 상승하고 방전 시에는 반대로 전압이 감...
...,...,...,...
198592,전기,Figure 1 shows the SEM images of bentonite and...,벤토나이트와 활성화 벤토나이트의 SEM 이미지를 Figure 1에 나타내었다.
198597,전기,It can also be seen that it is strongly attach...,또한 이것이 그라파이트 입자들과도 강하게 부착되어 있는 것을 볼 수 있다.
198599,전기,Heat transfer and reaction rate between gas an...,유동층 반응기 내 기체와 고체 간의 열전달 및 반응속도는 열전달 특성에 영향을 받는다.
198601,전기,"Meanwhile, the medical error index is periodic...",한편 모니터링 모듈을 통해 의료 오류지수가 주기적으로 관리된다.


In [23]:
electron_df = df['domain'] == '전자'

electron_df = df[electron_df]

electron_df

,domain,mt,ko
5,전자,"If a specific bit of the bitmap is 1, the RBG ...",비트맵의 특정 비트가 1이면 해당 RBG는 그 제1타입 단말에게 할당된 것으로 해석...
12,전자,The slit area 200 may include a plurality of s...,상기 슬릿 영역(200)은 복수개의 슬릿(231)을 포함할 수 있다.
20,전자,"In one example, audio chunks of the intermedia...","일 예에서, 중간 스트림의 오디오 청크들은 중간 스트림의 비디오 청크들보다 클 수 있다."
21,전자,The base station transmits control information...,기지국은 608단계에서 상기 선택된 송신 빔을 이용하여 제어정보를 전송한다.
32,전자,The processor 1113 may control the overall ope...,프로세서(1113)는 기지국 장치(1110) 전반의 동작을 제어할 수 있다.
...,...,...,...
1195103,전자,"Specifically, the thickness d3 of the insulato...","구체적으로, 상기 절연체(110)의 두께(d3)는 100 마이크로미터 이하로 이루어..."
1195159,전자,"In one embodiment, the input switch SWin may b...","일 실시예에서, 입력 스위치(SWin)는 클럭 신호(CLK)에 응답하여 온/오프될 ..."
1195167,전자,"In order to prevent this, a binder is needed t...","이를 방지하기 위해 용융된 상태에서도, 공융염을 형태를 유지하고 지지해줄 바인더가 ..."
1195193,전자,Such a light emitting display device has a pro...,이러한 발광 표시 장치는 복원 잔상으로 인하여 화질이 저하되는 문제점이 있다.


In [24]:
electron_df_1= electron_df.iloc[:20000, :]
electron_df_1

,domain,mt,ko
5,전자,"If a specific bit of the bitmap is 1, the RBG ...",비트맵의 특정 비트가 1이면 해당 RBG는 그 제1타입 단말에게 할당된 것으로 해석...
12,전자,The slit area 200 may include a plurality of s...,상기 슬릿 영역(200)은 복수개의 슬릿(231)을 포함할 수 있다.
20,전자,"In one example, audio chunks of the intermedia...","일 예에서, 중간 스트림의 오디오 청크들은 중간 스트림의 비디오 청크들보다 클 수 있다."
21,전자,The base station transmits control information...,기지국은 608단계에서 상기 선택된 송신 빔을 이용하여 제어정보를 전송한다.
32,전자,The processor 1113 may control the overall ope...,프로세서(1113)는 기지국 장치(1110) 전반의 동작을 제어할 수 있다.
...,...,...,...
199465,전자,It was found that SOx is efficiently controlle...,순산소 연소를 통한 운전 중 탈황 시설에서 SOx가 효율적으로 제어되고 수은도 효과...
199468,전자,In the case of the charging factor (FF) and po...,"충전율(FF)과 발전 효율(Eff.)의 경우, 3% 이내로 감소하는 변화를 관측할 ..."
199474,전자,"In other words, the electronic device 101 may ...","다시 말해, 전자 장치(101)는 해당 액션에 대해 수정 및 삭제의 권한을 가지고 ..."
199511,전자,Figure 5 shows the concentration profile of th...,Figure 5에 내부 공비제를 이용한 실험의 반응 증류의 농도 프로필을 나타내었다.


In [25]:
tech_df = df['domain'] == '기계'

tech_df = df[tech_df]

tech_df

,domain,mt,ko
0,기계,All skin corrosive/irritant substances contain...,오븐 세척제에 포함된 피부 부식성/자극성 물질은 모두 심한 눈손상성/눈 자극성에도 ...
4,기계,"In addition, it is possible to determine wheth...",센싱에 의해 공기 중 오일입자가 기준치이하인지 여부를 판단할 수 있다.
7,기계,The heat pipe 100 is a heat transport device h...,히트파이프(100)는 고 열전도성 금속에 비해 열전도성이 수십 배에서 수백 배 큰 ...
9,기계,"At the medical institution level, it should be...","의료기관 차원에서 위험도에 따라 고위험, 중위험, 저위험 의료기기로 분류하여야 한다."
13,기계,The controller 140 adjusts the vibration inten...,제어부(140)는 검출된 깊이에 대응하여 진동모터(131)의 진동 세기를 조정한다.
...,...,...,...
1195213,기계,"As an amino acid-type amphoteric surfactant, a...","아미노산형 양성 계면활성제로서는, 예를 들면 이하의 식(XI)으로 나타내는 계면활성..."
1195217,기계,AAA applied an alternating current of 800 μA a...,AAA는 실험체에 15분 동안 식염수주입을 주입하는 동안 피실험자의 팔에 800 μ...
1195218,기계,The initiator solution may be an aqueous solut...,"상기 개시제 용액은 수용액일 수 있고, 상기 전도성 고분자 중합 개시 물질로 철 이..."
1195222,기계,Various treatment methods based on various pri...,다양한 원리에 기초한 여러 가지 처리 방법이 개발되어 왔으며 산 또는 알칼리 가수 ...


In [26]:
tech_df_1= tech_df.iloc[:20000, :]
tech_df_1

,domain,mt,ko
0,기계,All skin corrosive/irritant substances contain...,오븐 세척제에 포함된 피부 부식성/자극성 물질은 모두 심한 눈손상성/눈 자극성에도 ...
4,기계,"In addition, it is possible to determine wheth...",센싱에 의해 공기 중 오일입자가 기준치이하인지 여부를 판단할 수 있다.
7,기계,The heat pipe 100 is a heat transport device h...,히트파이프(100)는 고 열전도성 금속에 비해 열전도성이 수십 배에서 수백 배 큰 ...
9,기계,"At the medical institution level, it should be...","의료기관 차원에서 위험도에 따라 고위험, 중위험, 저위험 의료기기로 분류하여야 한다."
13,기계,The controller 140 adjusts the vibration inten...,제어부(140)는 검출된 깊이에 대응하여 진동모터(131)의 진동 세기를 조정한다.
...,...,...,...
85318,기계,1 is a view showing the appearance of a vehicl...,도 1은 본 발명에 따른 차량용 변속장치를 포함하는 차량의 외관을 도시한 도면이다.
85319,기계,The surface of the upper operation housing 242...,상기 상부조작하우징(242)의 표면은 빛을 선택적으로 투과할 수 있는 재질이 코팅될...
85322,기계,"In general, a compressor is a device that comp...","일반적으로 압축기는 냉장고, 공기조화기 등의 냉각 장치에서 증발된 냉매를 압축하는 ..."
85327,기계,These fibers produced 5 to 30 mV under the con...,이 섬유는 시누소이드 자극 상태에서 5~30 mV를 생성했다.


In [27]:
med_df = df['domain'] == '의학'

med_df = df[med_df]

med_df

,domain,mt,ko
2,의학,"Venous blood was collected before surgery, and...",수술 전 정맥혈을 채취하고 월경주기의 난포기 동안 8시간 이상 금식한 후 정맥혈액을...
3,의학,Only 8.2% of those accredited for grade 3 or l...,3등급 이하 인정자는 8.2%만이 재가급여에서 시설급여로 변경하였다.
10,의학,Caution should be exercised in interpreting th...,이와 같은 신장 사건에 대한 연구 결과를 해석함에 있어 주의해야 할 점이다.
14,의학,Assess the correlation between the subject's s...,"대상자의 성 지식, 성 태도와 HPV 예방접종 이행 의도 간의 상관관계를 파악한다."
15,의학,There were no specific findings in the rheumat...,류마티스 인자와 항핵항체 검사에서 특이 소견은 없었고 환자는 류마티스 질환 관련 증...
...,...,...,...
1195214,의학,"They differ from group 1 ILCs, including NK ce...",이들은 NK1.1을 표현하지 않기 때문에 NK 셀을 포함한 1그룹 ILC와는 다르다.
1195215,의학,Here we report a 21-year-old soldier with a he...,여기에서 우리는 아버지의 유전에 의해 제안 된 폐 혈전 색전증과 폐 경색증을 보이는...
1195216,의학,"Meanwhile, the indications for tracheostomy in...",한편 소아에서 기관절개술의 적응증은 나이에 따른 차이를 보인다.
1195219,의학,"During the same period, the prevalence of obes...",같은 기간 동안 비만의 유병률이 급격히 증가했으며 전 세계적으로 주요 공중 보건 문...


In [28]:
med_df_1= med_df.iloc[:20000, :]
med_df_1

,domain,mt,ko
2,의학,"Venous blood was collected before surgery, and...",수술 전 정맥혈을 채취하고 월경주기의 난포기 동안 8시간 이상 금식한 후 정맥혈액을...
3,의학,Only 8.2% of those accredited for grade 3 or l...,3등급 이하 인정자는 8.2%만이 재가급여에서 시설급여로 변경하였다.
10,의학,Caution should be exercised in interpreting th...,이와 같은 신장 사건에 대한 연구 결과를 해석함에 있어 주의해야 할 점이다.
14,의학,Assess the correlation between the subject's s...,"대상자의 성 지식, 성 태도와 HPV 예방접종 이행 의도 간의 상관관계를 파악한다."
15,의학,There were no specific findings in the rheumat...,류마티스 인자와 항핵항체 검사에서 특이 소견은 없었고 환자는 류마티스 질환 관련 증...
...,...,...,...
60309,의학,Incontinence is a common health problem in soc...,요실금은 사회에서 흔한 건강 문제이고 노화에 따라 늘어난다.
60316,의학,"The internal health control committee, the oth...","내적건강통제위, 타인건강통제위, 우연건강통제위는 각각 높은 수준, 중간 수준, 낮은..."
60321,의학,The meal was a Korean standard meal consisting...,"식사는 탄수화물 60%, 지방 25%, 단백질 15%로 구성된 한국 표준 식사였다."
60322,의학,"In countries with weak state enforcement, it w...",국가의 강제성이 약한 나라들에서는 국민들이 주로 개인 연금을 통해 노후를 준비하고 ...


In [29]:
new_df = pd.concat([ict_df_1, ele_df_1, electron_df_1, tech_df_1, med_df_1])
new_df

,domain,mt,ko
6,ICT,The predetermined input may be an input that f...,미리 정해진 입력은 필기 입력의 일부를 포함하는 폐곡선을 형성하는 입력일 수 있다.
8,ICT,According to another aspect of the present inv...,"본 발명의 또 다른 양상에 따르면, 휴대 장치를 제어하는 장치가 제공된다."
11,ICT,The storage device 100 may communicate with th...,스토리지 장치(100)는 다양한 인터페이스를 통하여 호스트(200)와 통신할 수 있다.
16,ICT,Self-employment can be seen as a very importan...,"고용인 4명 중 1명꼴로 자영업에 의존하고 있는 만큼, 자영업은 우리에게 매우 중요..."
17,ICT,"Although not shown in the drawing, a discharge...","도면에는 도시되지 않았으나, 제1 및 제2 노드들(N1 및 N2)에는 디스차지 회로..."
...,...,...,...
60309,의학,Incontinence is a common health problem in soc...,요실금은 사회에서 흔한 건강 문제이고 노화에 따라 늘어난다.
60316,의학,"The internal health control committee, the oth...","내적건강통제위, 타인건강통제위, 우연건강통제위는 각각 높은 수준, 중간 수준, 낮은..."
60321,의학,The meal was a Korean standard meal consisting...,"식사는 탄수화물 60%, 지방 25%, 단백질 15%로 구성된 한국 표준 식사였다."
60322,의학,"In countries with weak state enforcement, it w...",국가의 강제성이 약한 나라들에서는 국민들이 주로 개인 연금을 통해 노후를 준비하고 ...


In [30]:
new_df.to_excel('D:/Jupyter/ah_bert/BERT_AutoencoderVA_2/기술과학_Data_10만.xlsx', index=False)